In [1]:
from PIL import Image, ImageOps
import torch
from torch import Tensor, linalg
import numpy as np
from IPython.display import Latex
import matplotlib.pyplot as plt
import os

import sklearn

from sklearn import cluster, metrics

import pandas as pd

# Generate the Data Matrix and the Label vector
Read data into torch.Tensor

In [2]:
torch.set_default_dtype(torch.float64)

# Specify the top-level folder
top_folder = "./data"

# Initialize an empty list to store flattened arrays
flattened_arrays = []
labels = torch.zeros(9120)
example_cnt, example_label = 0, 0

for root, dirs, files in os.walk(top_folder):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)
            lines = []
            with open(file_path, "r") as file:
                for line in file:
                    values = line.strip().split(",")
                    lines.append([float(value) for value in values])

            flattened_array = torch.tensor(lines).view(-1)
            labels[example_cnt] = example_label

            flattened_arrays.append(flattened_array)
            example_cnt += 1

            if example_cnt % 480 == 0:
                example_label += 1

# all_data_1 is a 2D tensor of shape (9120, 5625) containing 45 x 125 features for each data point
all_data_1 = torch.stack(flattened_arrays)

# all_data_2 is a 2D tensor of shape (9120, 45) containing the mean of each column in each segment resulting in 45 features for each data point
all_data_2 = torch.zeros((all_data_1.shape[0], 45))

for i in range(all_data_2.shape[1]):
    all_data_2[:, i] = all_data_1[:, i * 125 : i * 125 + 125].mean(1)

tensor([ 0.,  0.,  0.,  ..., 18., 18., 18.])


In [11]:
all_data_tst = torch.zeros((19 * 60 * 8, 45 * 125))
labels = torch.zeros(19 * 60 * 8)

top_folder = "./data"
i = 0
curr_label = 0
for root, dirs, files in os.walk(top_folder):
    if len(dirs) != 0 or root == top_folder:
        continue

    for file in files:
        file_path = os.path.join(root, file)

        df = pd.read_csv(file_path, header=None)
        person_data = torch.tensor(df.values)

        std = person_data.std(0)
        mean = person_data.mean(0)

        person_data = (person_data - mean) / std
        all_data_tst[i] = person_data.reshape(-1)

        labels[i] = curr_label

        i += 1
        if i % 480 == 0:
            curr_label += 1

all_data_2 = torch.zeros((all_data_1.shape[0], 45))

for i in range(all_data_2.shape[1]):
    all_data_2[:, i] = all_data_1[:, i * 125 : i * 125 + 125].mean(1)

print(all_data_tst.shape)

torch.Size([9120, 5625])


# Split the Dataset into Training and Test sets
Split dataset into training and test data taking the even indexed rows for testing and the odd indexed rows for training

In [3]:
training_indices = [i for i in range(len(all_data_1)) if (i % 60) < 48]
test_indices = [i for i in range(len(all_data_1)) if (i % 60) >= 48]

training_data_1 = all_data_1[training_indices]
test_data_1 = all_data_1[test_indices]

training_data_2 = all_data_2[training_indices]
test_data_2 = all_data_2[test_indices]

training_labels = labels[training_indices]
test_labels = labels[test_indices]

# Clustering Using K-Means and Normalized Cut

In [4]:
def k_means(points: Tensor, k: int, relative_error: float = 1e-6, max_iterations: int = 1000):
    n = len(points)
    means = points[torch.randperm(n)[:k]]

    error = 1
    iterations = 0
    while error > relative_error and iterations < max_iterations:
        iterations += 1
        distances = torch.cdist(points, means)
        closest_means = torch.argmin(distances, dim=1)

        new_means = torch.zeros_like(means)
        for i in range(k):
            new_means[i] = points[closest_means == i].mean(dim=0)

        error = torch.norm(means - new_means) / torch.norm(new_means)

        means = new_means

    return means, closest_means

In [11]:
def k_means_2(points:Tensor,k):
    
    '''
        points: dataset
        k: number of clusters
    '''
    plen = len(points[0]) if points.ndim > 1 else 1
    means = points[torch.randperm(len(points))[:k]]
    itr = 0
    while True:
        itr+=1
        classes = [[] for _ in range(k)]
        
        for j,point in enumerate(points):
            min_distance = torch.inf
            nearest_mean = None
        
            for i in range (len(means)):
                distance = torch.linalg.norm(point - means[i])
                if distance < min_distance:
                    min_distance = distance
                    nearest_mean = i
            
            classes[nearest_mean].append((j+1,point))
           
        new_means = torch.tensor([[0.0 for _ in range(plen)] for _ in range(k)]) 
        for i,c in enumerate(classes):
            n = float(len(c))
            new_mean = torch.tensor([0.0 for _ in range(plen)])
            for j,point in c:
                new_mean+=point
            new_mean = new_mean/n
            new_means[i] = new_mean
        
        if(torch.equal(new_means,means)):
            return classes
        
        means = new_means

### solution1:Taking the mean of each column in each segment for each data point

### solution2:Flattening all the features together for each data point

In [16]:
ks = [8, 13, 19, 28,38]

a = torch.cdist(all_data_tst, all_data_tst)
for k in ks:
    # means, mylabels = k_means(all_data_2, k)
    centroids, mylabels, inertia = cluster.k_means(all_data_tst.numpy(), k)
    # mylabels = cluster.k_means(a, k)
    print(metrics.f1_score(labels, mylabels, average='macro'))
    print(f1_score(Tensor(mylabels), labels))
    
        # classes = k_means_2(all_data_1,k)
        
        # for i in range(len(classes)):
        #     classes[i] = [classes[i][j][0] for j in range(len(classes[i]))]
        
        # mylabels = np.zeros(19)
        # for i,c in  enumerate(classes):
        #     for idx in c:
        #         mylabels[idx-1] = i
            
        #accuracy = torch.sum(labels == mylabels)/len(labels)
        
        #print(f'accuracy for k = {k} is {accuracy}')

centroids, mylabels, inertia = cluster.k_means(all_data_2.numpy(), 19)

for l in mylabels:
    print(l)

print(Tensor(mylabels))

0.019232697088924697
0.2188774010944789
0.021014126725182675
0.28606526300205676
0.029080841511425717
0.27479506114141755
0.01549427600185919
0.2700093309245396
0.020992712446163568
0.24000115165769687


NameError: name 'all_data_2' is not defined

# Evaluation

## Precision

In [6]:
def precision(clustering: Tensor, labels: Tensor):
    cluster_labels = torch.unique(clustering)

    total_precision = 0.0

    for cluster_label in cluster_labels:
        cluster_indices = (clustering == cluster_label).nonzero()

        actual_cluster_labels = labels[cluster_indices]
        mode = actual_cluster_labels.mode(dim=0)[0]
        total_precision += len(actual_cluster_labels[actual_cluster_labels == mode])

    return total_precision / len(clustering)

In [7]:
def recall(clustering: Tensor, labels: Tensor):
    cluster_labels = torch.unique(clustering)

    total_recall = 0.0

    for cluster_label in cluster_labels:
        cluster_indices = (clustering == cluster_label).nonzero()

        actual_cluster_labels = labels[cluster_indices]
        mode = actual_cluster_labels.mode(dim=0)[0]
        total_recall += len(actual_cluster_labels[actual_cluster_labels == mode]) / len(labels[labels == mode])

    return total_recall / len(cluster_indices)

In [13]:
def f1_score(clustering: Tensor, labels: Tensor):
    cluster_labels = torch.unique(clustering)

    total_f = 0.0

    for cluster_label in cluster_labels:
        cluster_indices = (clustering == cluster_label).nonzero()

        actual_cluster_labels = labels[cluster_indices]
        mode = actual_cluster_labels.mode(dim=0)[0]

        precision = len(actual_cluster_labels[actual_cluster_labels == mode]) / len(cluster_indices)
        recall = len(actual_cluster_labels[actual_cluster_labels == mode]) / len(labels[labels == mode])
        total_f += 2 * precision * recall / (precision + recall)

    return total_f / len(cluster_labels)

In [ ]:
def conditional_entropy(clustering: Tensor, labels: Tensor):
    cluster_labels = torch.unique(clustering)
    partition_labels = torch.unique(labels)

    total_entropy = 0.0
    for cluster_label in cluster_labels:
        cluster_entropy = 0.0
        cluster_indices = (clustering == cluster_label).nonzero()

        for partition_label in partition_labels:
            partition_indices = (labels == partition_label).nonzero()
            cluster_entropy -= len(partition_indices) / len(cluster_indices) * torch.log2(torch.Tensor(len(partition_indices) / len(cluster_indices)))
        
        total_entropy += 1 / len(cluster_indices) * cluster_entropy

    return total_entropy

In [172]:
f1_score(Tensor(mylabels), labels)

0.2832752154123034

## K-Ways normalised Cut

In [79]:
def rbf_graph(data: Tensor, gamma: float):
    graph = torch.zeros((len(data), len(data)))
    for i in range(len(data)):
        for j in range(len(data)):
            graph[i][j] = torch.exp(-gamma * torch.norm(data[i] - data[j]) ** 2)

    return graph

In [10]:
def k_ways_normalised_cut(a: Tensor, k: int):
    delta = a.sum(dim=1).diag()
    l_a = (1 / delta).diag().diag() @ (delta - a)
    
    eigen_values, eigen_vectors = torch.linalg.eig(l_a)
    eigen_values = eigen_values.real
    eigen_vectors = eigen_vectors.real

    indices = torch.argsort(eigen_values)
    eigen_values = eigen_values[indices]
    eigen_vectors = eigen_vectors[:, indices]

    u = eigen_vectors[:, :k]
    y = u / torch.norm(u, dim=1, keepdim=True)

    means, closest_means = k_means(y, k)

    return closest_means

In [17]:
# a = rbf_graph(all_data_2, 0.1)

a = torch.cdist(all_data_2, all_data_2)
delta = a.sum(dim=1).diag()
w = (1 / delta).diag().diag() @ a

# mylabels = k_ways_normalised_cut(a, 19)
# f1_score(mylabels, labels)

# metrics.f1_score(labels, mylabels, average='weighted')

NameError: name 'all_data_2' is not defined

In [65]:
def power_method(a: Tensor, k: int):
    n = len(a)
    v = torch.randn(n)
    v /= torch.norm(v)

    for _ in range(1000):
        v = a @ v
        v /= torch.norm(v)a

    return k_means(v[:, None], k)

In [66]:
means, mylabels = power_method(w, 19)
f1_score(mylabels, labels)

0.1